In [1]:
import pandas as pd
import fasttext
import pymorphy3
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

In [2]:
lenta = pd.read_csv('lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
lenta.head()

,title,text,topic
0,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика
1,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика
2,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика
3,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика
4,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика


In [3]:
lenta.shape

(258273, 3)

In [4]:
lenta.topic.value_counts() # таргет задачи

topic
Экономика          79528
Спорт              64413
Культура           53797
Наука и техника    53136
Бизнес              7399
Name: count, dtype: int64

In [16]:
lenta['title'].str.lower()

0                телеканалы станут вещать по единому тарифу
1                 volkswagen выкупает остатки акций "шкоды"
2                  прибыль тюменнефтегаза возросла в 10 раз
3         крупнейшее в истории слияние компаний происход...
4                   газ получил четверть обещанного кредита
                                ...                        
258268    российский боец ufc включен в книгу рекордов г...
258269    бывший чемпион ufc не выдержал кровопролития и...
258270               моуринью сравнил футболистов с мебелью
258271          путин предостерег от запретов рэп-концертов
258272    падение горнолыжника на полной скорости попало...
Name: title, Length: 258273, dtype: object

In [17]:
m = pymorphy3.MorphAnalyzer()

regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [18]:
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "

In [19]:
def clean_text(text):
    return lemmatize(words_only(text))

In [22]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, lenta['title']), total=len(lenta)))
    
lenta['lemmas'] = lemmas
lenta.head()

/tmp/ipykernel_5591/1996037650.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, lenta['title']), total=len(lenta)))


  0%|          | 0/258273 [00:00<?, ?it/s]

,title,text,topic,lemmas
0,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,телеканал стать вещать по единый тариф
1,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,volkswagen выкупать остаток акция шкода
2,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,прибыль тюменнефтегаз возрасти в раз
3,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,крупный в история слияние компания происходить...
4,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,газ получить четверть обещать кредит


In [23]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, lenta['topic']), total=len(lenta)))

lenta['topic_lem'] = lemmas
lenta.head()

/tmp/ipykernel_5591/282211254.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, lenta['topic']), total=len(lenta)))


  0%|          | 0/258273 [00:00<?, ?it/s]

,title,text,topic,lemmas,topic_lem
0,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,телеканал стать вещать по единый тариф,экономика
1,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,volkswagen выкупать остаток акция шкода,экономика
2,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,прибыль тюменнефтегаз возрасти в раз,экономика
3,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,крупный в история слияние компания происходить...,экономика
4,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,газ получить четверть обещать кредит,экономика


In [30]:
X = lenta.lemmas.tolist()
y = lenta.topic_lem.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 173042
total test examples 85231


In [31]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [32]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

Read 1M words
Number of words:  43803
Number of labels: 5
Progress: 100.0% words/sec/thread:  168033 lr:  0.000000 avg.loss:  0.167099 ETA:   0h 0m 0s


In [33]:
dir(fasttext)

['BOW',
 'EOS',
 'EOW',
 'FastText',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'absolute_import',
 'cbow',
 'division',
 'load_model',
 'print_function',
 'skipgram',
 'supervised',
 'tokenize',
 'train_supervised',
 'train_unsupervised',
 'unicode_literals']

In [34]:
result

(85231, 0.9575858549119451, 0.9575858549119451)

In [35]:
print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.9575858549119451
R@1: 0.9575858549119451
Number of examples: 85231


In [41]:
X_test

array(['спасти авиакомпания сша от банкротство мочь миллиард доллар',
       'тарпищев вернуть кузнецов в сборная россия по теннис',
       'персонаж смешарик спародировать сериал в всё тяжкий', ...,
       'евросоюз задуматься о продление кредит для греция на год',
       'глава нью йоркский биржа грозить отставка из за премия в миллион доллар',
       'эколог просчитать риск проект строительство северный поток'],
      dtype='<U119')

In [48]:
!head -n 10 test.txt

__label__экономика спасти авиакомпания сша от банкротство мочь миллиард доллар
__label__спорт тарпищев вернуть кузнецов в сборная россия по теннис
__label__культура персонаж смешарик спародировать сериал в всё тяжкий
__label__экономика росатом построить аэс акковать с новый партнёр
__label__наука и техника наса показать гигантский пузырь
__label__экономика чистый прибыль сибнефть за год вырасти в два раз
__label__культура михаил горбачёв встретиться в кремль с гельмут колоть для съёмка телесериал
__label__спорт тренер манчестер юнайтед отправить руни в дубль
__label__наука и техника предсказать срок тотальный ожирение человечество
__label__экономика эксперт обсудить экология теплоснабжение красноярск


In [49]:
!head -n 10 data.train.txt

__label__экономика курс доллар снизиться до рубль
__label__экономика при строительство подмосковный кольцо наслать нарушение на миллиард рубль
__label__спорт давид вилья прилететь на матч на частный самолёт
__label__культура неизвестный песня the doors издать в январь год
__label__спорт избитый футболист сборная россия болельщик уличить в ложь
__label__экономика бельгийский конкурент mcdonald s открыть ресторан в москва
__label__экономика в сша ужесточить правило выдача ипотечный кредит
__label__культура голландский музей идентифицировать картина халс
__label__культура лидер crowded house записать альбом с участник radiohead
__label__наука и техника зенитный ракета смочь поражать надводный цель


In [50]:
classifier.predict("лидер crowded house записать альбом с участник radiohead")

(('__label__культура',), array([1.00000131]))